# encoder 侧定义

In [3]:
from utils import *

In [4]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        """
        搭建整个Transformer的Encoder，主要是初始化编码器层和编码层的个数
        :param layer:(nn.Module)单层Encoder
        :param N: (int)单层Encoder重复的次数
        """
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        """
        前向传播
        :param x:
        :param mask:
        :return:
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        """
        单个Encoder层。Encoder 由 self-attn 和 feed forward 组成 (后面再定义)单个Encoder层。
        :param size: (int)模型尺寸(embeding_dim)
        :param self_attn: (nn.Module)自注意力层
        :param feed_forward: (nn.Module)前向传播层
        :param dropout: (int)丢弃机制
        """
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        # 只有 2 个encoder block 连接
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        """
        前向传播，参考论文的连接方式(这里迷惑的可以看 utils 中的注释)
        :param x: 输入向量
        :param mask:
        :return:
        """
        # 第一层处理的是多头注意力结果
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # 第二层计算的是加入前馈神经网络的残差输出
        return self.sublayer[1](x, self.feed_forward)